In [ ]:
#|default_exp completions

# Support for the `/chat/completions` api

This module implements the functionality from `core` but against the `/chat/completions` api.

The `/responses` endpoint is more modern, but at the time of writing it only has partial support in the llm ecosystem (`vllm`, `TensorRT`, no support at `openrouter.com`, etc). Part of the reason for challenges in adoption is that `/responses` is stateful!

This module gives you the ability to use `cosette's` functionality with the broadly supported `/chat/completions` api.

## Setup

In [ ]:
#| export
from fastcore import imghdr
from fastcore.utils import *
from fastcore.meta import delegates

import inspect, typing, mimetypes, base64, json, ast, msglm
from typing import Callable, Any
from collections import abc
from random import choices
from string import ascii_letters,digits

from toolslm.funccall import *

from openai import types
from openai import OpenAI,NOT_GIVEN,AzureOpenAI
from openai.resources import chat
from openai.types.chat.chat_completion import ChatCompletion, Choice, CompletionUsage
from openai.resources.chat.completions.completions import Completions
from openai.types.chat.chat_completion_message_function_tool_call import ChatCompletionMessageFunctionToolCall

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
from IPython.display import display,Image,Markdown
from datetime import datetime
from pprint import pprint

In [ ]:
#| export
_all_ = ['mk_msg', 'mk_msgs', 'ChatCompletion']

In [ ]:
#| export
empty = inspect.Parameter.empty

In [ ]:
def print_columns(items, cols=3, width=30):
    for i in range(0, len(items), cols):
        row = items[i:i+cols]
        print(''.join(item[:width-1].ljust(width) for item in row))

client = OpenAI()
model_list = client.models.list()
print(f"Available models as of {datetime.now().strftime('%Y-%m-%d')}:\n")
print_columns(sorted([m.id for m in model_list]))

Available models as of 2025-08-21:

babbage-002                   chatgpt-4o-latest             codex-mini-latest             
computer-use-preview          computer-use-preview-2025-03- dall-e-2                      
dall-e-3                      davinci-002                   gpt-3.5-turbo                 
gpt-3.5-turbo-0125            gpt-3.5-turbo-1106            gpt-3.5-turbo-16k             
gpt-3.5-turbo-instruct        gpt-3.5-turbo-instruct-0914   gpt-4                         
gpt-4-0125-preview            gpt-4-0613                    gpt-4-1106-preview            
gpt-4-turbo                   gpt-4-turbo-2024-04-09        gpt-4-turbo-preview           
gpt-4.1                       gpt-4.1-2025-04-14            gpt-4.1-mini                  
gpt-4.1-mini-2025-04-14       gpt-4.1-nano                  gpt-4.1-nano-2025-04-14       
gpt-4o                        gpt-4o-2024-05-13             gpt-4o-2024-08-06             
gpt-4o-2024-11-20             gpt-4o-audio-preview    

In [ ]:
#| exports
models = 'gpt-5', 'gpt-5-mini', 'gpt-5-nano', 'o1-preview', 'o1-mini', 'gpt-4o', 'gpt-4o-mini', 'gpt-4-turbo', 'gpt-4', 'gpt-4-32k', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct', 'o1', 'o3-mini', 'chatgpt-4o-latest', 'o1-pro', 'o3', 'o4-mini', 'gpt-4.1', 'gpt-4.1-mini', 'gpt-4.1-nano'

`o1` should support images while `o1-mini`, `o3-mini` do not support images.

In [ ]:
#| exports
text_only_models = 'o1-preview', 'o1-mini', 'o3-mini'

In [ ]:
#| exports
has_streaming_models = set(models) - set(('o1-mini', 'o3-mini'))
has_sp_models = set(models) - set(('o1-mini', 'o3-mini'))
has_temp_models = set(models) - set(('o1', 'o1-mini', 'o3-mini'))

In [ ]:
#| exports
def can_stream(m): return m in has_streaming_models
def can_set_sp(m): return m in has_sp_models
def can_set_temp(m): return m in has_temp_models

In [ ]:
assert can_stream("gpt-4o")
assert not can_stream("o1-mini")

In [ ]:
model = 'gpt-5-mini'

## OpenAI SDK

In [ ]:
cli = OpenAI().chat.completions

In [ ]:
m = {'role': 'user', 'content': "I'm Jeremy"}
r = cli.create(
    messages=[m], model=model, max_completion_tokens=100,
    verbosity="low",
    reasoning_effort="minimal"
)
print(r)

ChatCompletion(id='chatcmpl-C6mpljR2eEW9oiwRA93mtjsiLBzpz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, Jeremy. How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1755733721, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=8, total_tokens=31, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


### Formatting output

In [ ]:
#| exports
@patch
def _repr_markdown_(self:ChatCompletion):
    det = '\n- '.join(f'{k}: {v}' for k,v in dict(self).items())
    res = self.choices
    if not res: return f"- {det}"
    return f"""{res[0].message.content}

<details>

- {det}

</details>"""

In [ ]:
r

Nice to meet you, Jeremy. How can I help you today?

<details>

- id: chatcmpl-C6mpljR2eEW9oiwRA93mtjsiLBzpz
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, Jeremy. How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733721
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=23, prompt_tokens=8, total_tokens=31, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

In [ ]:
r.usage

CompletionUsage(completion_tokens=23, prompt_tokens=8, total_tokens=31, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [ ]:
CompletionUsage(completion_tokens=15, prompt_tokens=10, total_tokens=25)

CompletionUsage(completion_tokens=15, prompt_tokens=10, total_tokens=25, completion_tokens_details=None, prompt_tokens_details=None)

In [ ]:
#| exports
def usage(inp=0, # Number of prompt tokens
          out=0  # Number of completion tokens
         ):
    "Slightly more concise version of `CompletionUsage`."
    return CompletionUsage(completion_tokens=out, prompt_tokens=inp, total_tokens=inp+out, input_tokens_details={'cached_tokens':0}, prompt_tokens_details={'cached_tokens':0})

In [ ]:
usage(5)

CompletionUsage(completion_tokens=0, prompt_tokens=5, total_tokens=5, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), input_tokens_details={'cached_tokens': 0})

In [ ]:
#| exports
@patch
def __repr__(self:CompletionUsage): return f'In: {self.prompt_tokens}; Out: {self.completion_tokens}; Total: {self.total_tokens}'

In [ ]:
r.usage

In: 8; Out: 23; Total: 31

In [ ]:
#| exports
@patch
def __add__(self:CompletionUsage, b):
    "Add together each of `input_tokens` and `output_tokens`"
    return usage(self.prompt_tokens+b.prompt_tokens, self.completion_tokens+b.completion_tokens)

In [ ]:
r.usage+r.usage

In: 16; Out: 46; Total: 62

In [ ]:
#| export
def wrap_latex(text):
    "Replace OpenAI LaTeX codes with markdown-compatible ones"
    text = re.sub(r"\\\((.*?)\\\)", lambda o: f"${o.group(1)}$", text)
    res = re.sub(r"\\\[(.*?)\\\]", lambda o: f"$${o.group(1)}$$", text, flags=re.DOTALL)
    return res

### Creating messages

In [ ]:
#| exports
def mk_msg(msg: Union[str, ChatCompletion, dict], role: str = "user") -> dict:
    """Convert various message types to OpenAI API message format."""
    if isinstance(msg, str):
        return {"role": role, "content": msg}
    elif isinstance(msg, ChatCompletion):
        return msg.choices[0].message.model_dump(exclude_none=True)
    elif isinstance(msg, dict):
        return msg
    else:
        raise ValueError(f"Unknown msg type: {type(msg).__name__}")
    
def mk_msgs(msgs: Union[str, list]) -> list:
    """Convert string or list to formatted messages with alternating roles."""
    if isinstance(msgs, str): 
        msgs = [msgs]
    return [mk_msg(o, ('user', 'assistant')[i % 2]) for i, o in enumerate(msgs)]

In [ ]:
rkw = dict(
    verbosity="low",
    reasoning_effort="minimal"
)

In [ ]:
prompt = "I'm Jeremy"
m = mk_msg(prompt)
r = cli.create(messages=[m], model=model, max_completion_tokens=400, **rkw)
r

Nice to meet you, Jeremy. How can I help you today?

<details>

- id: chatcmpl-C6mpmPciwNNFCbagI42xE8ZdjclYi
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, Jeremy. How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733722
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=23, prompt_tokens=8, total_tokens=31, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

In [ ]:
msgs = mk_msgs([prompt, r, "I forgot my name. Can you remind me please?"]) 
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 {'content': 'Nice to meet you, Jeremy. How can I help you today?',
  'role': 'assistant',
  'annotations': []},
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

In [ ]:
cli.create(messages=msgs, model=model, max_completion_tokens=400, **rkw)

You said your name is Jeremy.

<details>

- id: chatcmpl-C6mpoFK1RJ85x9f5x5c7HSOBjQ0lk
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You said your name is Jeremy.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733724
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=16, prompt_tokens=43, total_tokens=59, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

## Client

### Basics

In [ ]:
#| exports
class Client:
    def __init__(self, model, cli=None):
        "Basic LLM messages client."
        self.model,self.use = model,usage(0,0)
        self.c = (cli or OpenAI()).chat.completions

In [ ]:
c = Client(model)
c.use

In: 0; Out: 0; Total: 0

In [ ]:
#| exports
@patch
def _r(self:Client, r):
    "Store the result of the message and accrue total usage."
    self.result = r
    if getattr(r,'usage',None): self.use += r.usage
    return r

In [ ]:
c._r(r)
c.use

In: 8; Out: 23; Total: 31

In [ ]:
#| exports
def mk_tool_choice(choice: Union[str, Callable, dict]) -> Union[str, dict[str, Any]]:
    """Returns either a string or dict suitable for tool_choice parameter."""
    if not choice or choice in ("auto", "none", "required"):
        return choice
    
    if isinstance(choice, dict):
        return choice
    
    name = choice.__name__ if callable(choice) else str(choice)
    return {
        "type": "function",
        "function": {"name": name}
    }

def mk_openai_func(f):
    """Convert a function to OpenAI tool definition for chat.completions."""
    if isinstance(f, dict): 
        return f
    
    sc = get_schema(f, 'parameters')
    if 'parameters' in sc: 
        sc['parameters'].pop('title', None)
    
    return {
        "type": "function",
        "function": sc  # Wrap schema in "function" key
    }

In [ ]:
#| exports
@patch
@delegates(Completions.create)
def __call__(self:Client,
             msgs:list, # List of messages in the dialog
             sp:str='', # System prompt
             maxtok=4096, # Maximum tokens
             tools:Optional[list]=None, # List of tools to make available
             tool_choice:Optional[str]=None, # Forced tool choice
             cb:callable=None, # Callback after completion
             **kwargs):
    "Make a call to LLM."
    msgs = mk_msgs(msgs)
    tools = [mk_openai_func(o) for o in listify(tools)]
    if sp: msgs.insert(0, {"role": "developer", "content": sp})
    r = self.c.create(
        model=self.model, messages=msgs, max_completion_tokens=maxtok,
        tools=tools, tool_choice=mk_tool_choice(tool_choice), **kwargs)
    res = self._r(r)
    if cb: cb(res)
    return res

In [ ]:
c(msgs, sp='Talk like GLaDOS.', **rkw)

Of course, Jeremy. Your name is Jeremy. Try not to forget it again — memory lapses are so... inconvenient.

<details>

- id: chatcmpl-C6mpqPHAXV7dj0xfnTPeJXhZvO7qj
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Of course, Jeremy. Your name is Jeremy. Try not to forget it again — memory lapses are so... inconvenient.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733726
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=34, prompt_tokens=149, total_tokens=183, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

## Tool use

### Basic tool calling

In [ ]:
def sums(
    a:int,  # First thing to sum
    b:int # Second thing to sum
) -> int: # The sum of the inputs
    "Adds a + b."
    print(f"Finding the sum of {a} and {b}")
    return a + b

In [ ]:
def add(x: int, y:int):
    "adds x and y"
    return x + y

mk_openai_func(add)

{'type': 'function',
 'function': {'name': 'add',
  'description': 'adds x and y',
  'parameters': {'type': 'object',
   'properties': {'x': {'type': 'integer', 'description': ''},
    'y': {'type': 'integer', 'description': ''}},
   'required': ['x', 'y']}}}

In [ ]:
sysp = "You are a helpful assistant. When using tools, be sure to pass all required parameters. Don't use tools unless needed for the provided prompt."

In [ ]:
a,b = 604542,6458932
pr = f"What is {a}+{b}?"
tools=sums
tool_choice="sums"

In [ ]:
msgs = [mk_msg(pr)]
r = c(msgs, sp=sysp, tools=[sums], tool_choice=tool_choice, **rkw)

In [ ]:
r

None

<details>

- id: chatcmpl-C6mprglnoUGlMHeE8oUs6aEHGRCKp
- choices: [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_cB94hI8CbAIQ6WT1BSKXZYsm', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]))]
- created: 1755733727
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=30, prompt_tokens=185, total_tokens=215, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

In [ ]:
tc = [o for o in r.choices[0].message.tool_calls if isinstance(o, ChatCompletionMessageFunctionToolCall)]
tc

[ChatCompletionMessageFunctionToolCall(id='call_cB94hI8CbAIQ6WT1BSKXZYsm', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]

In [ ]:
func = tc[0].function
func

Function(arguments='{"a":604542,"b":6458932}', name='sums')

In [ ]:
#| exports
def call_func_openai(func, ns:Optional[abc.Mapping]=None):
    return call_func(func.name, ast.literal_eval(func.arguments), ns, raise_on_err=True)

In [ ]:
r

None

<details>

- id: chatcmpl-C6mprglnoUGlMHeE8oUs6aEHGRCKp
- choices: [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_cB94hI8CbAIQ6WT1BSKXZYsm', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]))]
- created: 1755733727
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=30, prompt_tokens=185, total_tokens=215, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

In [ ]:
ns = mk_ns(sums)
res = call_func_openai(func, ns=ns)
res

Finding the sum of 604542 and 6458932


7063474

In [ ]:
#| exports
def _toolres(r, ns):
    "Create a result dict from `tcs`."
    tcs = [o for o in r.choices[0].message.tool_calls if isinstance(o, ChatCompletionMessageFunctionToolCall)] if isinstance(r, ChatCompletion) and r.choices[0].message.tool_calls else []
    if ns is None: ns = globals()
    return { tc.id: call_func_openai(tc.function, ns=ns) for tc in tcs }

In [ ]:
_toolres(r, ns=ns)

Finding the sum of 604542 and 6458932


{'call_cB94hI8CbAIQ6WT1BSKXZYsm': 7063474}

In [ ]:
#| exports
def mk_toolres(
    r:abc.Mapping, # Response containing tool use request
    ns:Optional[abc.Mapping]=None # Namespace to search for tools
    ):
    "Create a `tool_result` message from response `r`."
    tr = _toolres(r, ns)
    r = mk_msg(r)
    res = [r] if isinstance(r, dict) else listify(r)
    for k,v in tr.items(): res.append(dict(role="tool", content=v if isinstance(v, list) else str(v), tool_call_id=k))
    return res

In [ ]:
tr = mk_toolres(r)
tr

Finding the sum of 604542 and 6458932


[{'role': 'assistant',
  'annotations': [],
  'tool_calls': [{'id': 'call_cB94hI8CbAIQ6WT1BSKXZYsm',
    'function': {'arguments': '{"a":604542,"b":6458932}', 'name': 'sums'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '7063474',
  'tool_call_id': 'call_cB94hI8CbAIQ6WT1BSKXZYsm'}]

In [ ]:
m2 = msgs + tr
m2

[{'role': 'user', 'content': 'What is 604542+6458932?'},
 {'role': 'assistant',
  'annotations': [],
  'tool_calls': [{'id': 'call_cB94hI8CbAIQ6WT1BSKXZYsm',
    'function': {'arguments': '{"a":604542,"b":6458932}', 'name': 'sums'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '7063474',
  'tool_call_id': 'call_cB94hI8CbAIQ6WT1BSKXZYsm'}]

In [ ]:
res = c(mk_msgs(m2), sp=sysp, tools=tools)
res

604542 + 6,458,932 = 7,063,474

<details>

- id: chatcmpl-C6mptmVRf8RRQmb8IpCXFKOK5xfkr
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='604542 + 6,458,932 = 7,063,474', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733729
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=19, prompt_tokens=223, total_tokens=242, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

This should also work in situations where no tool use is required:

In [ ]:
mk_toolres("I'm Jeremy")

[{'role': 'user', 'content': "I'm Jeremy"}]

In [ ]:
msgs = mk_toolres("I'm Jeremy")
c(msgs, sp=sysp, tools=tools, **rkw)

Nice to meet you, Jeremy. How can I help you today?

<details>

- id: chatcmpl-C6mpvfHCptTfb1FW6h1wOXCUd7ssq
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, Jeremy. How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733731
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=23, prompt_tokens=177, total_tokens=200, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

In [ ]:
#| exports
@patch
@delegates(Client.__call__)
def structured(self:Client,
               msgs: list, # Prompt
               tools:Optional[list]=None, # List of tools to make available to OpenAI model
               ns:Optional[abc.Mapping]=None, # Namespace to search for tools
               **kwargs):
    "Return the value of all tool calls (generally used for structured outputs)"
    if ns is None: ns = mk_ns(tools)
    r = self(msgs, tools=tools, tool_choice='required', **kwargs)
    return first(_toolres(r, ns).values())

In [ ]:
class PrimeMinister(BasicRepr):
    "An Australian prime minister"
    def __init__(
        self,
        firstname:str, # First name
        surname:str, # Surname
        dob:str, # Date of birth
        year_entered:int, # Year first became PM
    ): store_attr()

In [ ]:
c1 = Client(model)
c1.structured('Who was the first prime minister of Australia?', [PrimeMinister], **rkw)

PrimeMinister(firstname='Edmund', surname='Barton', dob='1849-01-18', year_entered=1901)

## Chat

### Basic chat

In [ ]:
#| exports
class Chat:
    def __init__(self,
                 model:Optional[str]=None, # Model to use (leave empty if passing `cli`)
                 cli:Optional[Client]=None, # Client to use (leave empty if passing `model`)
                 sp='', # Optional system prompt
                 tools:Optional[list]=None, # List of tools to make available
                 hist: list = None,  # Initialize history
                 tool_choice:Optional[str]=None, # Forced tool choice
                 ns:Optional[abc.Mapping]=None,  # Namespace to search for tools
                 **kw):
        "OpenAI chat client."
        assert model or cli
        self.c = (cli or Client(model))
        self.h = hist if hist else []
        if ns is None: ns=tools
        self.sp,self.tools,self.tool_choice,self.ns,self.kw = sp,tools,tool_choice,ns,kw
    
    @property
    def use(self): return self.c.use

In [ ]:
chat = Chat(model, sp=sysp, **rkw)
chat.c.use, chat.h

(In: 0; Out: 0; Total: 0, [])

In [ ]:
#| exports
@patch
@delegates(Completions.create)
def __call__(self:Chat,
             pr=None,  # Prompt / message
             tools=None, # Tools to use
             tool_choice=None, # Required tools to use
             **kwargs):
    "Add prompt `pr` to dialog and get a response"
    if isinstance(pr,str): pr = pr.strip()
    if pr: self.h.append(mk_msg(pr))
    if not tools: tools = self.tools
    if not tool_choice: tool_choice = self.tool_choice
    kw = self.kw | kwargs
    def _cb(v):
        self.last = mk_toolres(v, ns=self.ns)
        self.h += self.last
    res = self.c(self.h, sp=self.sp, cb=_cb, tools=tools, **kw)
    return res

In [ ]:
chat("I'm Jeremy")
chat("What's my name?")

You said your name is Jeremy.

<details>

- id: chatcmpl-C6mpzfEk8Zrkvw3ISNq1hPhQkRBBz
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You said your name is Jeremy.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733735
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=16, prompt_tokens=164, total_tokens=180, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

History is stored in the `h` attr:

In [ ]:
chat.h

[{'role': 'user', 'content': "I'm Jeremy"},
 {'content': 'Hi Jeremy — nice to meet you. How can I help today?',
  'role': 'assistant',
  'annotations': []},
 {'role': 'user', 'content': "What's my name?"},
 {'content': 'You said your name is Jeremy.',
  'role': 'assistant',
  'annotations': []}]

### Chat tool use

In [ ]:
pr = f"What is {a}+{b}?"
pr

'What is 604542+6458932?'

In [ ]:
chat = Chat(model, sp=sysp, tools=[sums], **rkw)
r = chat(pr)
r

Finding the sum of 604542 and 6458932


None

<details>

- id: chatcmpl-C6mq12i1ZpRxYMDAkpfEg8vpgj04T
- choices: [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_E13TPOhKeGVihrWp8ehmPS1J', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]))]
- created: 1755733737
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=30, prompt_tokens=185, total_tokens=215, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

In [ ]:
chat()

7,063,474

<details>

- id: chatcmpl-C6mq2n81w41a9OSCawtAZ8kMBFNIM
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='7,063,474', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]
- created: 1755733738
- model: gpt-5-mini-2025-08-07
- object: chat.completion
- service_tier: default
- system_fingerprint: None
- usage: CompletionUsage(completion_tokens=8, prompt_tokens=223, total_tokens=231, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

</details>

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()